# Algorithme du gradient conjugué

In [1]:
using LinearAlgebra
using Optim

In [2]:
x = [ 1 ; 0 ]
iter = reshape(copy(x),:,1)
sizeof(iter)
x = [ 1 ; 1 ]
iter = hcat(iter, x)
iter

2×2 Matrix{Int64}:
 1  1
 0  1

Résolvez
$$
\min_x f(x) = \frac{1}{2} x^T A x + b^T x + a
$$
où $A \succ 0$. En posant $\nabla f(x) = 0$, c'est équivalent à résoudre le système linéaire $Ax = -b$.

Construisons la fonction quadratique associée au programme précédent.

In [3]:
f = x -> 0.5*dot(x,A*x)+dot(b,x)

#2 (generic function with 1 method)

## Un exemple simple

Adapté de https://www.rose-hulman.edu/~bryan/lottamath/congrad.pdf

Soit
$$
A =
\begin{pmatrix}
3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4
\end{pmatrix}
$$
Considérons la fonction à minimiser
$$
f(x) = \frac{1}{2} x^TAx,
$$
et supposons que nous avons déjà calculé
\begin{align*}
d_0 &= (1, 0, 0)\\
d_1 & = (1, −3, 0)\\
d_2 &= (−2, 6, −5).
\end{align*}

Vérifions que $d_0$, $d_1$ et $d_2$ sont $A$-conjugés.

In [4]:
A = [ 3.0 1 0 ; 1 2 2 ; 0 2 4]
d0 = [ 1.0 0 0 ]'
d1 = [ 1.0 -3.0 0.0 ]'
d2 = [ -2.0 6.0 -5.0]'

println("$(dot(d0, A*d1)) $(dot(d0, A*d2)) $(dot(d1, A*d2))")

0.0 0.0 0.0


Vérifions que les valeurs propres de $A$ sont positives, et donc que le problème est convexe.

In [5]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 0.47108204270564347
 3.167449191108536
 5.361468766185826
vectors:
3×3 Matrix{Float64}:
 -0.325306   0.916757  0.231804
  0.822673   0.15351   0.547398
 -0.466246  -0.368771  0.804128

Prenons comme solution initiale $x_0 = (1, 2, 3)$. Calculons $x_1$, $x_2$ et $x_3$ en utilisant l'algorithme du gradient conjugué. $x_3$ est-il optimal?

Nous avons
$$
\nabla f(x) = Ax
$$

Nous définissons la fonction objectif comme suit.

In [6]:
f = x -> dot(x,A*x)

#5 (generic function with 1 method)

Nous devons calculer $\alpha_k$, $k = 0,1,2$, en résolvant
$$
\min_{\alpha} f(x_k + \alpha d_k)
$$

Afin d'obtenir $\alpha_0$, nous devons minimiser
\begin{align*}
f(x_0 + \alpha d_0) &= \frac{1}{2}
\left(\begin{pmatrix} 1 & 2 & 3\end{pmatrix} + \alpha \begin{pmatrix} 1 & 0 & 0\end{pmatrix} \right)
\begin{pmatrix}
3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4
\end{pmatrix}
\left(\begin{pmatrix} 1 \\ 2 \\ 3 \end{pmatrix} + \alpha \begin{pmatrix} 1 \\0 \\0 \end{pmatrix} \right)
\\
& = \frac{1}{2}\begin{pmatrix} 1 + \alpha & 2 & 3 \end{pmatrix}
\begin{pmatrix}
3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4
\end{pmatrix}
\begin{pmatrix} 1 + \alpha \\ 2 \\ 3 \end{pmatrix}\\
& = \frac{1}{2}\begin{pmatrix} 1 + \alpha & 2 & 3 \end{pmatrix}
\begin{pmatrix} 5+3\alpha \\ 11+\alpha \\ 16 \end{pmatrix}\\
& = \frac{1}{2}
((1 + \alpha)(5+3\alpha) + 22+2\alpha + 48 ) \\
& = \frac{1}{2}
( 3\alpha^2 + 8\alpha + 5 + 70 + 2\alpha ) \\
& = \frac{3}{2}\alpha^2 + 5\alpha+\frac{75}{2}
\end{align*}
par rapport à $\alpha$.

Nous pouvons l'obtenir en cherchant le zéro de la dérivée par rapport à $\alpha$, c'est-à-dire
$$
\frac{d}{d\alpha} f(x+\alpha d) = 0,
$$
ou

$$
d^T \nabla f(x+\alpha d) = 0
$$

Dès lors, nous devons avoir

$$
3\alpha + 5 = 0
$$
Ainsi,
$$
\alpha_{0} = -\frac{5}{3}
$$
$$
x_1 = x_0 - \frac{5}{3} d_0 = \begin{pmatrix} -\frac{2}{3} \\ 2 \\ 3  \end{pmatrix}
$$

Nous pouvons aussi directement calculer $\alpha_0$ comme
$$
\alpha_0 = - \frac{d_0^T\nabla f(x_0)}{d_0^TAd_0}
$$

In [7]:
x0 = [1 ; 2 ; 3.0]
∇f = x -> A*x
d0 = [1 ; 0 ; 0]
α0 = -dot(d0,∇f(x0))/dot(d0,A*d0)

-1.6666666666666667

Calculons le nouvel itéré.

In [8]:
x1 = x0+α0*d0

3-element Vector{Float64}:
 -0.6666666666666667
  2.0
  3.0

Une recherche linéaire à partir de $x_1$ dans la direction $d_1$ exige de minimiser
\begin{align*}
f(x_1 + \alpha d_1) & = \left(\begin{pmatrix} -\frac{2}{3} & 2 & 3 \end{pmatrix} + \alpha_1\begin{pmatrix} 1 & -3 & 0 \end{pmatrix} \right)\begin{pmatrix} 3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4 \end{pmatrix}\left(\begin{pmatrix}  -\frac{2}{3} \\ 2 \\ 3 \end{pmatrix} +  \alpha_1\begin{pmatrix} 1 \\ -3 \\ 0 \end{pmatrix} \right) \\
& =\frac{15}{2}\alpha^2 - 28\alpha + \frac{100}{3},
\end{align*}
ce qui a lieu en
$$
\alpha_1 = \frac{28}{15},
$$
donnant
$$
x_2 = x_1 + \frac{28}{15}d_1 =
    \begin{pmatrix}
     \frac{6}{5} \\ \frac{-18}{5} \\ 3
    \end{pmatrix}.
$$

La nouvelle longueur de pas est

In [9]:
α1 = -dot(d1,A*x1)/dot(d1,A*d1)

1.8666666666666665

ou, sous forme de fraction, 28/15:

In [10]:
28/15

1.8666666666666667

Le nouvel itéré est

In [11]:
x2 = x1+α1*d1

3×1 Matrix{Float64}:
  1.1999999999999997
 -3.5999999999999996
  3.0

La recherche linéaire finale à partir de $x_2$ dans la direction $d_2$ requiert de minimiser
$$
f(x_2 + \alpha d_2) = 20 \alpha^2 - 24\alpha + \frac{36}{5},
$$
ce qui a lieu en
$$
\alpha_2 = \frac{3}{5},
$$
donnant
$$
x_3 = x_2 + \frac{3}{5}d_2 =
    \begin{pmatrix}
     0 \\ 0 \\ 0
    \end{pmatrix},
$$
ce qui est bien entendu correct.

Similairement, nous pouvons calculer le nouveau point comme

In [12]:
α2 = -dot(d2,A*x2)/dot(d2,A*d2)
x3 = x2+α2*d2

3×1 Matrix{Float64}:
 -4.440892098500626e-16
  8.881784197001252e-16
 -4.440892098500626e-16

$x_3$ est clairement optimal comme $f(x_3) = 0$ et 0 est une borne inférieure sur $f(\cdot)$. Nous pouvons également vérifier que $x_3$ est un point critique. En effet

In [13]:
∇f(x3)

3×1 Matrix{Float64}:
 -4.440892098500626e-16
  4.440892098500626e-16
  0.0

## Une implémentation naïve

Une première version de l'algorithme du gradient conjugué suit.

In [14]:
function cg_quadratic(A:: Matrix, b:: Vector, x0:: Vector; trace:: Bool = false)
    n = length(x0)
    x = x0
    g = b+A*x
    d = -g
    if (trace)
        iter = reshape(copy(x),:,1)
        iterg = [ norm(g) ]
        iterd = [ norm(d) ]
    end
    k = 0
    
    for k = 1:n-1
        Ad = A*d
        normd = dot(d,Ad)
        α = -dot(d,g)/normd
        x += α*d
        if (trace)
            iter = hcat(iter, x)
            iterg = [ iterg; norm(g)]
            iterd = [ iterd; norm(d) ]
        end
        g = b+A*x
        β = dot(g,Ad)/normd
        d = -g+β*d
    end

    normd = dot(d,A*d)
    α = -dot(d,g)/normd
    x += α*d
    if (trace)
        g = b+A*x # g must be equal to 0
        iter = hcat(iter, x)
        iterg = [ iterg; norm(g)]
        iterd = [ iterd; norm(d) ]
        return x, iter, iterg, iterd
    end
    
    return x
end

cg_quadratic (generic function with 1 method)

Considérons l'exemple simple

In [15]:
A = [2 1; 1 2]
b = [1, 0]
A\(-b)

2-element Vector{Float64}:
 -0.6666666666666666
  0.3333333333333333

Nous voulons résoudre
$$
    \min_{\alpha} f(x) = \frac{1}{2}x^TAx+b^Tx+c
$$

Ou, de manière équivalente, nous résolvons
$$
    c+\min_{\alpha} f(x) = \frac{1}{2}x^TAx+b^Tx
$$

Appliquons l'algorithme que nous avons implémenté.

In [16]:
cg_quadratic(A, b, [0, 0], trace = true)

([-0.6666666666666666, 0.3333333333333333], [0.0 -0.5 -0.6666666666666666; 0.0 0.0 0.3333333333333333], [1.0, 1.0, 0.0], [1.0, 1.0, 0.5590169943749475])

Que se passe-t-il si $A$ n'est pas définie positive? Considérons l'exemple simple suivant:

In [17]:
A = [ 1 2 ; 2 1]
A\(-b)

2-element Vector{Float64}:
  0.3333333333333333
 -0.6666666666666666

In [18]:
cg_quadratic(A, b, [0, 0], trace = true)

([0.33333333333333326, -0.6666666666666666], [0.0 -1.0 0.33333333333333326; 0.0 0.0 -0.6666666666666666], [1.0, 1.0, 1.1102230246251565e-16], [1.0, 1.0, 4.47213595499958])

In [19]:
det(A)

-3.0

In [20]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
2-element Vector{Float64}:
 -1.0
  3.0
vectors:
2×2 Matrix{Float64}:
 -0.707107  0.707107
  0.707107  0.707107

In [21]:
cg_quadratic(A, b, [1, 1], trace = true)

([0.3333333333333335, -0.6666666666666667], [1.0 -0.36986301369863006 0.3333333333333335; 1.0 -0.02739726027397249 -0.6666666666666667], [5.0, 5.0, 2.220446049250313e-16], [5.0, 5.0, 0.9763790695367754])

La solution est $x^* = (1/3, -2/3)$. $f(x^*)$ vaut

In [22]:
f([1/3,-2/3])

-0.3333333333333333

Le gradient conjugué trouve la solution du système linéaire, laquelle correspond à un point critique au premier ordre de la fonction.

In [23]:
∇f = x -> A*x+b;

In [24]:
x = [1.0/3; -2.0/3]
∇f(x)

2-element Vector{Float64}:
 0.0
 0.0

Mais ce n'est pas un minimum de la fonction! Nous pouvons en effet aisément partir de ce point et diminuer la valeur de la fonction. Construisons la fonction de calcul d'un itéré le long de la plus forte pente.

In [25]:
step= x -> x-α*∇f(x);

Nous pouvons obtenir une direction de courbure négative en prenant le vecteur propre associé à une valeur propre négative.

In [26]:
λ, u = eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
2-element Vector{Float64}:
 -1.0
  3.0
vectors:
2×2 Matrix{Float64}:
 -0.707107  0.707107
  0.707107  0.707107

In [27]:
x = u[:,1] # premier vecteur propre associé à λ = -1
A*x

2-element Vector{Float64}:
  0.7071067811865475
 -0.7071067811865475

Remarquons que la norme du vecteur propre ainsi obtenu vaut 1.

In [28]:
1.0-norm(x)

1.1102230246251565e-16

Construison un point le long de cette direction.

In [29]:
α = 10
f = x -> 0.5*dot(x,A*x)+dot(b,x)
f(step(x))

-106.05992052357223

La valeur de la fonction en ce point est clairement plus basse que la solution construite par l'algorithme du gradient conjugué!

In [30]:
x = [1/3.0; -2/3]
f(x)

0.16666666666666666

Remarquons que l'algorithme du gradient conjugué est incapable de réduire la fonction en partant de la solution préalablement trouvée.

In [31]:
cg_quadratic(A, b, x, trace = true)

([NaN, NaN], [0.3333333333333333 NaN NaN; -0.6666666666666666 NaN NaN], [0.0, 0.0, NaN], [0.0, 0.0, NaN])

Nous devons incorporer un test sur $\nabla f(x_k)$!

### Un exemple plus complexe.

In [32]:
using Random

Random.seed!(42)

n = 500;
m = 600;
A = randn(n,m);
A = A * A';  # A is now a positive semi-definite matrix
A = A+I # A is positive definite
b = zeros(n)
for i = 1:n
  b[i] = randn()
end
x0 = zeros(n);

Nous pouvons directement calculer la solution du système linéaire.

In [33]:
b1 = A\(-b)

500-element Vector{Float64}:
 -0.0062258396116907965
 -0.005181127240103852
  0.015073887348628504
 -0.01560759718414364
  0.02053086586034354
 -0.012561972984245986
  0.0203382901384829
  0.009921484755129583
  0.025053494365504615
 -0.01422170959830383
 -0.019456487375742064
  0.0016405107629915434
  0.00998244732532377
  ⋮
 -0.049605850895340184
 -0.016399859737709244
 -0.0027004894873878895
 -0.015510165276327207
 -0.01043966310588118
 -0.0555862749004948
  0.00023275820843085532
  0.01408041721348296
  0.016047679165720585
  0.003259550536496433
 -0.0124171590089707
 -0.018583274686171235

Essayons à présent le gradient conjugué.

In [34]:
b2, iter, iterg, iterd = cg_quadratic(A, b, x0, trace = true);

À nouveau, nous avons obtenu une solution au système $Ax = b$.

Les solutions trouvées sont très proches.

In [35]:
norm(b1-b2)

1.1386909068879741e-15

Affichons l'historique des normes du gradient.

In [36]:
iterg

501-element Vector{Float64}:
 22.47596587347289
 22.47596587347289
 19.852321369391685
 17.668972344139238
 16.460785043199287
 14.418235124687062
 12.399230571763502
 10.43301131463277
  9.022617018735529
  7.5050388008870685
  6.84282932189519
  6.479519717530909
  5.718379218278424
  ⋮
  1.0374820338990994e-13
  1.0582503635003376e-13
  1.0092853224429132e-13
  9.831346771350367e-14
  1.0158396919759237e-13
  1.0157958673141029e-13
  9.082342643356438e-14
  1.0146762017974174e-13
  1.0406475930438119e-13
  9.839540098068094e-14
  1.060059899702659e-13
  1.0603537397344082e-13

Affichons à présent l'historique des normes de la direction de recherche.

In [37]:
iterd

501-element Vector{Float64}:
 22.47596587347289
 22.47596587347289
 26.48751984736536
 27.430376509418405
 28.943830352031615
 26.476629264241236
 23.176373371982866
 19.444655677336144
 17.114281157822123
 14.019346189313467
 13.514860772976007
 13.741419618172824
 12.134532090573732
  ⋮
  1.0738421647036762e-13
  1.0945447061086957e-13
  1.0338456063193303e-13
  9.933063371168414e-14
  1.0541542702642289e-13
  9.906657607750841e-14
  9.060789951192603e-14
  1.0216593748325766e-13
  1.0387674547779996e-13
  9.936312567501064e-14
  1.0932470416042246e-13
  1.1285233115117889e-13

Cela fonctionne, mais devons-nous vraiment faire 500 itérations? Nous serions satisfaits si nous sommes proches de la solution. Nous pouvons mesurer le résidu du système linéaire residual of the linear system
$$
r = b+Ax,
$$
ce qui n'est rien d'autre que le gradient de la fonction objectif du problème de minimisation quadratique.

Nous devons inclure un test de convergence dans la fonction.

In [38]:
function cg_quadratic_tol(A:: Matrix, b:: Vector, x0:: Vector; trace:: Bool = false, tol = 1e-6)
    n = length(x0)
    x = x0
    if (trace)
        iter = reshape(copy(x),:,1)
    end
    g = b+A*x
    d = -g
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    while ((dot(g,g) > tol2) && (k < n))
        Ad = A*d
        normd = dot(d,Ad)
        α = dot(g,g)/normd
#        α = -dot(d,g)/normd
        x += α*d
        if (trace)
            iter = hcat(iter, x)
        end
        g = b+A*x
        β = dot(g,Ad)/normd
        d = -g+β*d
        k += 1
    end

    if (trace)
        return x, iter, k
    end

    return x, k
end

cg_quadratic_tol (generic function with 1 method)

In [39]:
x, iter, k = cg_quadratic_tol(A, b, x0, trace = true)

([-0.0062258394372755104, -0.005181126592629329, 0.015073885577242851, -0.015607597543385665, 0.020530866781195672, -0.01256197321023359, 0.020338290943214276, 0.00992148514634991, 0.02505349277775968, -0.014221710531303874  …  -0.0027004902371467565, -0.015510165175905505, -0.010439662662634071, -0.055586276032773226, 0.00023275797992716463, 0.014080415748308229, 0.016047678591395675, 0.003259552552504535, -0.012417157152793956, -0.01858327472169429], [0.0 0.0024369108070299585 … -0.006225839548128013 -0.0062258394372755104; 0.0 0.0005721844857703452 … -0.0051811266036876375 -0.005181126592629329; … ; 0.0 0.0021895888486158067 … -0.012417156870666078 -0.012417157152793956; 0.0 -0.003193733323955644 … -0.01858327469234316 -0.01858327472169429], 149)

Le nombre d'itérations est à présent

In [40]:
k

149

ce qui est nettement moindre que la dimension du problème.

In [41]:
size(A)

(500, 500)

Sommes-nous proche de la solution?

In [42]:
norm(b1-x)

3.0950354835360327e-8

In [43]:
norm(b+A*iter[:,149])

1.070067837304389e-6

## Gradient conjugué préconditionné

Si le nombre de conditionnement est égal à 1, nous convergeons en une itération.

Rappelons que le nombre de conditionnement d'une matrice $A$ définie positive est donné par
$$
\kappa(A) = \frac{\lambda_{\max}}{\lambda_{\min}}.
$$
$\kappa(A) = 1$ ssi $A = \gamma I$. Dans ce cas
$$
A = \begin{pmatrix} \gamma & 0 & \cdots & 0 \\ 0 & \gamma & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \cdots & \gamma \end{pmatrix}.
$$
Observons que $\lambda_{\max} = \lambda_{\min} = \gamma$.

Le problème quadratique devient alors
$$
f(x) = \frac{1}{2}\gamma x^Tx + b^Tx.
$$

Son gradient est
$$
\nabla f(x) = \gamma x + b.
$$
Il s'annule si
$$
x = -\frac{b}{\gamma}.
$$

Soit $x_0$. L'algorithme du gradient conjugué donne comme première de recherche $d_0 = -\nabla f(x_0) = -\gamma x_0 - b$.

Nous avons aussi
$$
\alpha_0 = - \frac{d_0^T\nabla f(x_0)}{d_0^TAd_0} = \frac{\| d_0 \|^2}{\gamma \| d_0 \|^2} = \frac{1}{\gamma}.
$$

Le premier itéré donne
$$
x_1 = x_0 + \alpha_0 d_0 = x_0 + \frac{1}{\gamma} (-\gamma x_0 - b) = -\frac{b}{\gamma}.
$$
ce qui correspond bien à la solution!

Si la matrice $A$ est diagonale et tous les éléments de la diagonale sont identiques, la direction de plus forte pente donne le minimum global.

Une implémentation basique d'un algorithme de gradient préconditionné suit, où $M$ est l'inverse du préconditioneur à appliquer.

In [44]:
function pcg_quadratic_tol(A:: Matrix, b:: Vector, x0:: Vector, M:: Matrix;
                           trace:: Bool = false, tol = 1e-6)
    n = length(x0)
    x = x0
    if (trace)
        iter = reshape(copy(x), :, 1)
    end
    g = b+A*x
    v = M*g
    d = -v
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    gv = dot(g,v)
    while ((gv > tol2) && (k < n))
#    while ((dot(g,g) > tol2) && (k < n))
        Ad = A*d
        normd = dot(d,Ad)
        #gv = dot(g,v)
        α = gv/normd
        x += α*d
        if (trace)
            iter = hcat(iter, x)
        end
        g += α*Ad
        v = M*g
        gvold = gv
        gv = dot(g,v)
        β = gv/gvold
        d = -v+β*d
        k += 1
    end

    if (trace)
        return x, iter, k
    end

    return x, k
end

pcg_quadratic_tol (generic function with 1 method)

Vérifions tout d'abord qu'en l'absence de préconditionnement, nous obtenons pratiquement les mêmes itérés. Il peut y avoir de légères variations en raison de la manière dont est accumulée l'erreur.

Posons

In [45]:
M = zeros(n,n)+I
x, iter, k = pcg_quadratic_tol(A, b, x0, M, trace = true)

([-0.006225839411169457, -0.0051811266471045395, 0.015073885629498388, -0.015607597564259296, 0.020530866736620933, -0.012561973161622707, 0.020338291007527005, 0.009921485190188116, 0.025053492858376145, -0.014221710511283915  …  -0.0027004901222787343, -0.015510165285449375, -0.010439662605942174, -0.05558627602856801, 0.00023275802058159042, 0.014080415733171765, 0.016047678567543272, 0.0032595525769107524, -0.01241715720764479, -0.018583274671121197], [0.0 0.0024369108070299585 … -0.0062258395353545215 -0.006225839411169457; 0.0 0.0005721844857703452 … -0.0051811265717767 -0.0051811266471045395; … ; 0.0 0.0021895888486158067 … -0.012417156935069427 -0.01241715720764479; 0.0 -0.003193733323955644 … -0.018583274752453846 -0.018583274671121197], 148)

Nous affichons le nombre d'itérations et si les solutions diffèrent fortement.

In [46]:
k, norm(x-b1)

(148, 3.003591861272444e-8)

Comme attendu, l'algorithme affiche des performances très similaires.

Nous pouvons calculer les valeurs propres et le nombre de conditionnement de la matrice $A$.

In [47]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
500-element Vector{Float64}:
    6.001354994197266
    6.738041057858181
    6.989710063968306
    7.697013876056844
    8.507800677062212
    8.876237594028309
    9.858305804097244
   10.35353373818242
   10.657983531540566
   11.252642176724246
   11.60989683916678
   12.630365179022595
   12.7552722187328
    ⋮
 1924.0952190995877
 1957.226070870355
 1962.0741032339981
 1978.7797324004475
 1984.4429789149603
 2021.831465244454
 2035.6230452912862
 2048.5623509454354
 2086.143325944565
 2097.5381604567424
 2140.1749782491834
 2147.8290254262097
vectors:
500×500 Matrix{Float64}:
  0.0475891    -0.003653    -0.0197161    …   0.0204719    -0.0157036
  0.0267188     0.0367903    0.0419689       -0.0498996     0.0319362
 -0.0448637     0.0225979   -0.0262229       -0.0304667    -0.0171819
  0.00334682    0.0100408   -0.00246739      -0.0209598    -0.0151802
 -0.00337866   -0.074695     0.0189748        0.0617855     0.0512

In [48]:
cond(A)

357.89068093838034

In [49]:
A

500×500 Matrix{Float64}:
 573.025      17.4568     14.4568    …   14.4597    58.7528   -13.1933
  17.4568    605.698     -47.83          35.4941    49.6088    -7.56193
  14.4568    -47.83      610.8          -29.3652    -8.39599   26.0283
  34.2017      4.23307     7.25559      -29.6451    25.775    -52.7125
   9.57431    -0.109639    7.52114      -58.458    -16.0098    26.3033
   0.365724   32.7943     -1.42132   …   -3.82901  -14.55      -0.332799
  -9.85352   -12.4834    -17.4351        20.9079    -8.80454   25.373
  20.0817     48.9277    -11.1236       -25.2626   -17.3545    10.2658
 -34.3403    -11.309      13.1794         1.40417   10.5461    67.8022
  10.9654     -4.66029   -25.2091         3.25653   24.9973    -3.99513
   8.70926    17.9578     12.4032    …  -18.5662   -30.7839    41.4219
  10.1531    -48.6254     37.4594       -19.0201     8.51632  -10.4266
 -44.3754    -13.9702     -3.377          9.12243  -28.047     -0.117705
   ⋮                                 ⋱         

Essayons de construire un préconditionneur simple en utilisant l'inverse de la diagonale de la matrice $A$.

In [50]:
D = 1 ./diag(A)
M = Diagonal(D)

500×500 Diagonal{Float64, Vector{Float64}}:
 0.00174512   ⋅           ⋅         …   ⋅           ⋅           ⋅ 
  ⋅          0.00165099   ⋅             ⋅           ⋅           ⋅ 
  ⋅           ⋅          0.0016372      ⋅           ⋅           ⋅ 
  ⋅           ⋅           ⋅             ⋅           ⋅           ⋅ 
  ⋅           ⋅           ⋅             ⋅           ⋅           ⋅ 
  ⋅           ⋅           ⋅         …   ⋅           ⋅           ⋅ 
  ⋅           ⋅           ⋅             ⋅           ⋅           ⋅ 
  ⋅           ⋅           ⋅             ⋅           ⋅           ⋅ 
  ⋅           ⋅           ⋅             ⋅           ⋅           ⋅ 
  ⋅           ⋅           ⋅             ⋅           ⋅           ⋅ 
  ⋅           ⋅           ⋅         …   ⋅           ⋅           ⋅ 
  ⋅           ⋅           ⋅             ⋅           ⋅           ⋅ 
  ⋅           ⋅           ⋅             ⋅           ⋅           ⋅ 
 ⋮                                  ⋱                          
  ⋅           ⋅      

Malheureusement, dans ce cas, nous n'observons pas une amélioration du nombre de conditionnement.

In [51]:
B = M*A
cond(B)

356.9189510539403

Considérons une autre situation où $A$ est diagonale.

In [52]:
n = 1000;
A = zeros(n,n);
for i = 1:n
    A[i,i] = 10*rand()
end
b = zeros(n)
for i = 1:n
  b[i] = rand()
end
x0 = zeros(n)
cond(A)

1564.6913643718426

La solution que nous cherchons est

In [53]:
A\b

1000-element Vector{Float64}:
 0.11940870642103221
 0.040195103932515366
 0.03743207176192072
 0.09596743111464928
 0.015247480640838236
 0.04426362774949533
 0.25642584282269737
 0.09927816239440633
 0.12842889948881794
 0.000989967399592411
 0.06930566548606212
 0.0855788318490393
 0.025680911363577415
 ⋮
 0.008890161826283391
 0.14458956365917086
 0.0663919524596606
 0.00407207679242235
 0.3400037325826622
 0.010837881327806494
 0.1420791741420285
 0.005895380125527243
 0.4259500686209045
 0.34704936720552254
 0.020632046571407922
 0.2722423993405068

Sans préconditionnement, nous avons la séquence d'itérés

In [54]:
M = zeros(n,n)+I
x, iter, k = pcg_quadratic_tol(A, b, x0, M, trace = true)

([-0.11940869927513621, -0.04019509823203459, -0.03743206913355815, -0.0959674342631363, -0.015247481061602973, -0.044263627850409985, -0.25642586384456784, -0.09927816258008909, -0.12842890390177644, -0.0009899674085476313  …  -0.06639195796921714, -0.004072076602973669, -0.3400037449435983, -0.010837881054943297, -0.14207916933552153, -0.005895380064402162, -0.42595004856992613, -0.3470493741082865, -0.02063204767163965, -0.2722423764493205], [0.0 -0.07374640992993094 … -0.11940870302942262 -0.11940869927513621; 0.0 -0.07746870640116056 … -0.04019511252223306 -0.04019509823203459; … ; 0.0 -0.03049794343015068 … -0.02063204592398165 -0.02063204767163965; 0.0 -0.05227887270300126 … -0.27224237050999495 -0.2722423764493205], 134)

In [55]:
sizeof(iter)

1080000

Ceci est équivalent à la version de l'algorithme non-préconditionné.

In [56]:
x, iter, k = cg_quadratic_tol(A, b, x0, trace = true)

([-0.11940869927513627, -0.040195098232034605, -0.03743206913355812, -0.09596743426313638, -0.015247481061602956, -0.044263627850409944, -0.2564258638445676, -0.09927816258008912, -0.12842890390177644, -0.0009899674085476324  …  -0.06639195796921718, -0.004072076602973669, -0.3400037449435981, -0.010837881054943295, -0.1420791693355215, -0.005895380064402162, -0.4259500485699266, -0.3470493741082865, -0.020632047671639623, -0.2722423764493208], [0.0 -0.07374640992993094 … -0.11940870302942269 -0.11940869927513627; 0.0 -0.07746870640116056 … -0.0401951125222331 -0.040195098232034605; … ; 0.0 -0.03049794343015068 … -0.02063204592398163 -0.020632047671639623; 0.0 -0.05227887270300126 … -0.2722423705099952 -0.2722423764493208], 134)

Cependant, puisque $A$ est diagonal, un préconditionneur diagonal évident est $A^{-1}$ lui-même.

In [57]:
M = zeros(n,n)
for i = 1:n
    M[i,i] = 1/A[i,i]
end

Le nombre de préconditionnement de la matrice préconditionnée est bien entendu égal à 1.

In [58]:
cond(M*A)

1.0000000000000002

La théorie prédit alors que nous convergeons en une itération avec le gradient conjugué précontionné.

In [59]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, trace = true)

([-0.11940870642103224, -0.04019510393251537, -0.03743207176192073, -0.09596743111464931, -0.01524748064083824, -0.044263627749495334, -0.2564258428226974, -0.09927816239440636, -0.12842889948881797, -0.0009899673995924113  …  -0.06639195245966062, -0.004072076792422351, -0.34000373258266225, -0.010837881327806496, -0.14207917414202853, -0.005895380125527244, -0.4259500686209046, -0.3470493672055226, -0.02063204657140793, -0.27224239934050687], [0.0 -0.11940870642103224; 0.0 -0.04019510393251537; … ; 0.0 -0.02063204657140793; 0.0 -0.27224239934050687], 1)

Mais le cas diagonal n'est pas très intéressant, comme le système peut être résolut facilement, en traitant les variables indépendamment.

Considérons à présent un autre exemple.

In [60]:
A = zeros(n,n)+3*I
for i = 1:n-1
    A[i,i+1] = 1.4
    A[i+1,i] = 1.4
end
A

1000×1000 Matrix{Float64}:
 3.0  1.4  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.4  3.0  1.4  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.4  3.0  1.4  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.4  3.0  1.4  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.4  3.0  1.4  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.4  3.0  1.4  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.4  3.0  1.4     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.4  3.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.4     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

Les valeurs propres de la matrice sont

In [61]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
1000-element Vector{Float64}:
 0.20001378984134793
 0.20005515922956107
 0.20012410775715758
 0.20022063474500001
 0.20034473924231072
 0.2004964200266737
 0.2006756756040492
 0.20088250420879217
 0.20111690380366315
 0.20137887207985267
 0.20166840645700262
 0.20198550408323332
 0.20233016183516794
 ⋮
 5.7980144959167665
 5.798331593542997
 5.798621127920147
 5.798883096196337
 5.799117495791208
 5.7993243243959505
 5.799503579973327
 5.79965526075769
 5.799779365255
 5.799875892242843
 5.799944840770439
 5.799986210158653
vectors:
1000×1000 Matrix{Float64}:
 -0.000140286  -0.00028057   -0.000420851  …   0.00028057   0.000140286
  0.00028057    0.000561129   0.000841665      0.000561129  0.00028057
 -0.000420851  -0.000841665  -0.0012624        0.000841665  0.000420851
  0.000561129   0.00112217    0.00168303       0.00112217   0.000561129
 -0.0007014    -0.00140263   -0.00210351       0.00140263   0.0007014
  0.0008416

La matrice est donc bien définie positive et la solution du système est

In [62]:
A\(-b)

1000-element Vector{Float64}:
  0.04257124905076859
 -0.3561612494967965
  0.44232177414384205
 -0.6985541045396697
  0.5591290010428805
 -0.57348924969193
  0.46621796795042886
 -0.7266978760001267
  0.48653956718976227
 -0.4345703177644765
  0.43840872870580627
 -0.64698541432064
  0.3459373854155322
  ⋮
 -0.025160889346438488
  0.024436887218838
 -0.26669768975339103
  0.4688452989330671
 -0.7605259301001155
  0.717245380372757
 -0.8463504791950822
  0.5984629331699821
 -0.4502035906570915
  0.15077990193920424
 -0.10303035249571006
 -0.03956574653533955

L'algorithme du gradient conjugué donne

In [63]:
x, iter, k = cg_quadratic_tol(A, b, x0, trace = true)

([0.04257125323084248, -0.35616124664916227, 0.44232176974515647, -0.6985540964234387, 0.5591289780776869, -0.5734892401496092, 0.4662179694663175, -0.726697879974238, 0.4865395542954642, -0.4345702971472391  …  -0.26669769476700195, 0.4688453271769261, -0.760525962210652, 0.7172454081438162, -0.8463505222945318, 0.5984629759985127, -0.45020363535525365, 0.15077993995382297, -0.10303036529025265, -0.039565754252828614], [0.0 -0.07269107908366373 … 0.04257125273680231 0.04257125323084248; 0.0 -0.07636010849702882 … -0.3561612555251321 -0.35616124664916227; … ; 0.0 -0.030061509704358055 … -0.10303037482758724 -0.10303036529025265; 0.0 -0.05153074805498146 … -0.03956574573503335 -0.039565754252828614], 44)

Essayons à nouveau comme précondtionneur l'inverse de la diagonale.

In [64]:
M = zeros(n,n)
for i = 1:n
    M[i,i] = 1/A[i,i]
end

Comparons les nombres de conditionnement de la matrice avec et sans conditionnement.

In [65]:
cond(A), cond(M*A)

(28.99793166640786, 28.997931666407833)

Le conditionnement de la matrice n'a pas changé, aussi nous ne devrions pas voir de différence lors de l'application de l'algorithme du gradient conjugué.

In [66]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, trace = true)

([0.04257125273680225, -0.356161255525132, 0.44232177027250236, -0.6985541065350427, 0.5591289986939593, -0.5734892429790511, 0.4662179900819047, -0.7266978983779363, 0.4865395621551865, -0.4345702681108459  …  -0.26669770447902474, 0.4688453320898843, -0.7605259943878858, 0.7172454420719291, -0.8463505166017918, 0.5984629932782357, -0.4502036569930646, 0.15077995380805448, -0.10303037482758724, -0.039565745735033284], [0.0 -0.07269107908366373 … 0.042571248950691226 0.04257125273680225; 0.0 -0.0763601084970288 … -0.35616125455862835 -0.356161255525132; … ; 0.0 -0.03006150970435805 … -0.10303040169823596 -0.10303037482758724; 0.0 -0.05153074805498145 … -0.0395656976357877 -0.039565745735033284], 43)

De fait, il n'y a pas d'avantage notable. Notons de plus la structure de $A^{-1}$.

In [67]:
M = inv(A)

1000×1000 Matrix{Float64}:
  0.490553      -0.336899       0.231373      …   5.26731e-164  -2.45808e-164
 -0.336899       0.721926      -0.4958           -1.12871e-163   5.26731e-164
  0.231373      -0.4958         0.831055          1.89193e-163  -8.82901e-164
 -0.158901       0.340503      -0.570747         -2.92543e-163   1.3652e-163
  0.109129      -0.233848       0.391974          4.37685e-163  -2.04253e-163
 -0.0749471      0.160601      -0.269198      …  -6.45353e-163   3.01165e-163
  0.0514717     -0.110297       0.184878          9.45214e-163  -4.411e-163
 -0.0353494      0.0757488     -0.126969         -1.38011e-162   6.44049e-163
  0.0242771     -0.0520223      0.0871993         2.01216e-162  -9.39006e-163
 -0.0166729      0.0357276     -0.0598862        -2.93166e-162   1.36811e-162
  0.0114505     -0.0245368      0.0411283     …   4.26996e-162  -1.99265e-162
 -0.00786389     0.0168512     -0.0282458        -6.21827e-162   2.90186e-162
  0.00540072    -0.011573       0.019398

Peut-on exploiter la structure creuse des matrices? La librairie `SparseArrays` vient ici à notre secours.

In [68]:
using SparseArrays

sparse(A)

1000×1000 SparseMatrixCSC{Float64, Int64} with 2998 stored entries:
⎡⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⎦

In [69]:
sparse(M)

1000×1000 SparseMatrixCSC{Float64, Int64} with 1000000 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎦

Nous pouvons visualier que l'inversion transforme une matrice creuse en matrice dense. Ce n'est pas intéressant, même si, évidemment, nous n'avons à présent besoin que d'une itération avec l'algorithme du gradient conjugé.

In [70]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, trace = true)

([0.042571249050768666, -0.35616124949679673, 0.442321774143842, -0.6985541045396702, 0.5591290010428808, -0.5734892496919305, 0.466217967950429, -0.7266978760001268, 0.48653956718976243, -0.4345703177644767  …  -0.2666976897533911, 0.4688452989330673, -0.7605259301001157, 0.7172453803727571, -0.8463504791950824, 0.5984629331699823, -0.4502035906570916, 0.15077990193920437, -0.10303035249571016, -0.03956574653533957], [0.0 0.042571249050768666; 0.0 -0.35616124949679673; … ; 0.0 -0.10303035249571016; 0.0 -0.03956574653533957], 1)

Considérons l'exemple suivant.

In [71]:
n = 1000
A = zeros(n,n)+Diagonal([2+i*i for i=1:n])

for i = 1:n-1
    A[i,i+1] = 1
    A[i+1,i] = 1
end
A[n,1] = 1
A[1,n] = 1
A

1000×1000 Matrix{Float64}:
 3.0  1.0   0.0   0.0   0.0   0.0  …       0.0       0.0       0.0  1.0
 1.0  6.0   1.0   0.0   0.0   0.0          0.0       0.0       0.0  0.0
 0.0  1.0  11.0   1.0   0.0   0.0          0.0       0.0       0.0  0.0
 0.0  0.0   1.0  18.0   1.0   0.0          0.0       0.0       0.0  0.0
 0.0  0.0   0.0   1.0  27.0   1.0          0.0       0.0       0.0  0.0
 0.0  0.0   0.0   0.0   1.0  38.0  …       0.0       0.0       0.0  0.0
 0.0  0.0   0.0   0.0   0.0   1.0          0.0       0.0       0.0  0.0
 0.0  0.0   0.0   0.0   0.0   0.0          0.0       0.0       0.0  0.0
 0.0  0.0   0.0   0.0   0.0   0.0          0.0       0.0       0.0  0.0
 0.0  0.0   0.0   0.0   0.0   0.0          0.0       0.0       0.0  0.0
 0.0  0.0   0.0   0.0   0.0   0.0  …       0.0       0.0       0.0  0.0
 0.0  0.0   0.0   0.0   0.0   0.0          0.0       0.0       0.0  0.0
 0.0  0.0   0.0   0.0   0.0   0.0          0.0       0.0       0.0  0.0
 ⋮                            ⋮    ⋱ 

La matrice ainsi construite est particulièrement creuse. De telles structures apparaissent des divers problèmes physiques.

Nous l'avons construite de telle manière à avoir un très mauvais conditionnement.

In [72]:
cond(A)

372201.88311573525

Le taux de convergence prédit est très lent.

In [73]:
κ = cond(A)
(sqrt(κ)-1)/(sqrt(κ)+1)

0.9967271248797894

Mais à nouveau, l'inverse de la matrice est dense.

In [74]:
A^(-1)

1000×1000 Matrix{Float64}:
  0.353263     -0.0597876     0.00546288   …   3.53969e-13  -3.53262e-7
 -0.0597876     0.179363     -0.0163886       -5.99071e-14   5.97875e-8
  0.00546288   -0.0163886     0.092869         5.4738e-15   -5.46287e-9
 -0.00030412    0.000912359  -0.00517004      -3.04728e-16   3.04119e-10
  1.12747e-5   -3.38241e-5    0.00019167       1.12972e-17  -1.12747e-11
 -2.96856e-7    8.90567e-7   -5.04654e-6   …  -2.97449e-19   2.96855e-13
  5.82243e-9   -1.74673e-8    9.89813e-8       5.83407e-21  -5.82242e-15
 -8.82347e-11   2.64704e-10  -1.49999e-9      -8.84111e-23   8.82346e-17
  1.06319e-12  -3.18958e-12   1.80743e-11      1.06532e-24  -1.06319e-18
 -1.04243e-14   3.12729e-14  -1.77213e-13     -1.04451e-26   1.04243e-20
  8.47552e-17  -2.54265e-16   1.44084e-15  …   8.49246e-29  -8.4755e-23
 -5.80538e-19   1.74161e-18  -9.86915e-18     -5.81699e-31   5.80537e-25
  3.39506e-21  -1.01852e-20   5.7716e-20       3.40185e-33  -3.39505e-27
  ⋮                         

Essayons tout d'abord avec la matrice identité (autrement dit, nous n'appliquons pas de préconditionnement).

In [75]:
M = zeros(n,n)+I
x, iter, k = pcg_quadratic_tol(A, b, x0, M, trace = true)

([-0.10686713399300515, -0.047201628059102156, -0.0036239360800608875, -0.03843954032330223, -0.0030013889125384404, -0.006985570495690997, -0.007674449186479286, -0.01281740387879107, -0.001490126085417836, -7.762898066012389e-5  …  -1.242135303153609e-7, -4.26067162906945e-8, -6.298219268005767e-7, -9.90736135675547e-8, -7.04317922873731e-7, 2.1901251789861791e-7, -3.036102604331485e-7, -3.2268108945084997e-7, -1.536993620196928e-7, -1.5688328242826325e-7], [0.0 -1.115653551853396e-6 … -0.10684508691878523 -0.10686713399300515; 0.0 -1.1719653544635081e-6 … -0.04722216484273041 -0.047201628059102156; … ; 0.0 -4.6138027524865326e-7 … -1.5369587562042394e-7 -1.536993620196928e-7; 0.0 -7.908874489403843e-7 … -1.5515790445753074e-7 -1.5688328242826325e-7], 1000)

Le nombre maximum d'itérations est atteint, pour une précision restant moyenne. En effet, si nous comparons avec la solution recherchée, nous obtenons

In [76]:
xopt = A\(-b)
norm(x-xopt)

0.003252316254556436

Essayons à nouveau de préconditionner le système avec l'inverse de la diagonale.

In [77]:
M = zeros(n,n)
for i = 1:n
    M[i,i] = 1/A[i,i]
end
cond(A*M), cond(A)

(1.9263607324508687, 372201.88311573525)

Le conditionnement de la matrice préconditionnée est cette fois-ci nettement meilleur, suggérant que $M$ est un bon préconditionneur. Essayons done l'algorithme du gradient conjugué préconditionné.

In [78]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, trace = true)

([-0.10834159229304484, -0.045887056911176144, -0.005969581727505766, -0.038083766239335945, -0.0021538619439104337, -0.00723557191117042, -0.007876432231704157, -0.012674364451665233, -0.0018485524335799133, -5.259286217144727e-5  …  -1.114953025420961e-7, -3.20877173027251e-8, -6.298363333607426e-7, -9.907463479806973e-8, -7.040815275967083e-7, -1.9945093679059217e-8, -3.0348806800920694e-7, -3.2347935378974093e-7, -1.5369770560823646e-7, -1.545977631104635e-7], [0.0 -0.1087838374132552 … -0.10834165329965362 -0.10834159229304484; 0.0 -0.057137311292618634 … -0.04588709135401175 -0.045887056911176144; … ; 0.0 -1.3523324025065907e-7 … -1.536972727593845e-7 -1.5369770560823646e-7; 0.0 -2.3135031330359105e-7 … -1.546009917976205e-7 -1.545977631104635e-7], 6)

Non seulement, le nombre d'itérations est nettement inférieur, mais la précision est grandement améliorée:

In [79]:
norm(x-xopt)

7.477146831566554e-9

Le préconditionnement est cependant souvent utilisé pour produire une matrice proche de $A$, mais permettant de résoudre de manière plus efficace des systèmes linéaires. La version suivante de l'algorithme du gradient conjugué préconditionné.

Deux versions sont proposées, une avec une matrice régulière, une avec une matrice triangulaire inférieure, sachant que résoudre un système triangulaire est direct.

In [80]:
function pcg_quadratic(A:: Matrix, b:: Vector, x0:: Vector, M:: Matrix;
                       trace:: Bool = false, tol = 1e-6)
    n = length(x0)
    x = x0
    if (trace)
        iter = reshape(copy(x), :, 1)
    end
    g = b+A*x
    v = M\g    # l'application de M se fait à présent en calculant un système linéaire plutôt qu'une multiplication matricielle.
    d = -v
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    gv = dot(g,v)
    while ((gv > tol2) && (k <= n))
#    while ((dot(g,g) > tol2) && (k <= n))
        Ad = A*d
        normd = dot(d,Ad)
        #gv = dot(g,v)
        α = gv/normd
        x += α*d
        if (trace)
            iter = hcat(iter, x)
        end
        g += α*Ad
        v = M\g
        gvold = gv
        gv = dot(g,v)
        β = gv/gvold
        d = -v+β*d
        k += 1
    end

    if (trace)
        return x, iter, k
    end

    return x, k
end

function pcg_quadratic(A:: Matrix, b:: Vector, x0:: Vector, L:: LowerTriangular;
                       trace:: Bool = false, tol = 1e-6)
    n = length(x0)
    x = x0
    if (trace)
        iter = reshape(copy(x), :, 1)
    end
    g = b+A*x
    U = transpose(L)
    v = U\(L\g)    # l'application de M se fait à présent en calculant un système linéaire plutôt qu'une multiplication matricielle.
    d = -v
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    gv = dot(g,v)
    while ((gv > tol2) && (k <= n))
#    while ((dot(g,g) > tol2) && (k <= n))
        Ad = A*d
        normd = dot(d,Ad)
        #gv = dot(g,v)
        α = gv/normd
        x += α*d
        if (trace)
            iter = hcat(iter, x)
        end
        g += α*Ad
        v = U\(L\g)
        gvold = gv
        gv = dot(g,v)
        β = gv/gvold
        d = -v+β*d
        k += 1
    end

    if (trace)
        return x, iter, k
    end

    return x, k
end

pcg_quadratic (generic function with 2 methods)

Un préconditionneur populaire est la factorisation de Cholesky incomplète, qui reproduit la factorisation de Cholesky, mais en ne considérant que les positions où l'élément de la matrice initiale est non nul, avec de conserver le caractère creux (si présent), et en s'arrêtant prématurément si la matrice se révèle non définie positive.

Une première implémentation naïve est donnée ci-dessous.

In [81]:
"""
    ichol(A::Matrix)

Factorisation incomplète de Cholesky.
Maintient le pattern de A (les zéros restent zéros).
"""
function ichol(A::Matrix)
    n = size(A, 1)
    C = copy(A)  # Copier A au lieu de créer une matrice identité
    
    for k = 1:n
        # Calculer C[k,k]
        C[k, k] = sqrt(C[k, k])
        
        # Normaliser la colonne k
        for i = (k+1):n
            if C[i, k] != 0  # Seulement si non-zéro dans le pattern
                C[i, k] = C[i, k] / C[k, k]
            end
        end
        
        # Mettre à jour le reste de la matrice
        for j = (k+1):n
            for i = j:n
                if C[i, j] != 0  # Seulement si non-zéro dans le pattern
                    C[i, j] = C[i, j] - C[i, k] * C[j, k]
                end
            end
        end
    end
    
    return LowerTriangular(C)
end

ichol

Essayons d'abord avec la factorisation complète de A.

In [82]:
C = cholesky(A)
C.L

1000×1000 LowerTriangular{Float64, Matrix{Float64}}:
 1.73205    ⋅         ⋅          ⋅          …     ⋅           ⋅            ⋅ 
 0.57735   2.38048    ⋅          ⋅                ⋅           ⋅            ⋅ 
 0.0       0.420084  3.28991     ⋅                ⋅           ⋅            ⋅ 
 0.0       0.0       0.303959   4.23174           ⋅           ⋅            ⋅ 
 0.0       0.0       0.0        0.23631           ⋅           ⋅            ⋅ 
 0.0       0.0       0.0        0.0         …     ⋅           ⋅            ⋅ 
 0.0       0.0       0.0        0.0               ⋅           ⋅            ⋅ 
 0.0       0.0       0.0        0.0               ⋅           ⋅            ⋅ 
 0.0       0.0       0.0        0.0               ⋅           ⋅            ⋅ 
 0.0       0.0       0.0        0.0               ⋅           ⋅            ⋅ 
 0.0       0.0       0.0        0.0         …     ⋅           ⋅            ⋅ 
 0.0       0.0       0.0        0.0               ⋅           ⋅            ⋅ 
 0.0       

In [83]:
M = C.L*C.U

1000×1000 Matrix{Float64}:
 3.0   1.0          0.0          …       0.0       0.0   1.0
 1.0   6.0          1.0                  0.0       0.0   0.0
 0.0   1.0         11.0                  0.0       0.0   0.0
 0.0   0.0          1.0                  0.0       0.0  -8.67362e-19
 0.0   0.0          0.0                  0.0       0.0   0.0
 0.0   0.0          0.0          …       0.0       0.0   0.0
 0.0   0.0          0.0                  0.0       0.0   0.0
 0.0   0.0          0.0                  0.0       0.0   0.0
 0.0   0.0          0.0                  0.0       0.0   0.0
 0.0   0.0          0.0                  0.0       0.0   0.0
 0.0   0.0          0.0          …       0.0       0.0   0.0
 0.0   0.0          0.0                  0.0       0.0   1.2326e-32
 0.0   0.0          0.0                  0.0       0.0   0.0
 ⋮                               ⋱                      
 0.0   0.0          0.0                  0.0       0.0   0.0
 0.0   0.0          0.0                  0.0   

Si nous essayons d'appliquer l'algorithme du gradient conjugué préconditionné, nous voyons que nous convergeons en une itération, ce qui est logique vu que $M = A$.

In [84]:
x, iter, k = pcg_quadratic(A, b, x0, M, trace = true)

([-0.10834159685648033, -0.045887056976095, -0.005969577304884257, -0.03808376588970455, -0.002153865038031075, -0.007235570879054487, -0.00787643404305555, -0.01267436504270527, -0.0018485519425091102, -5.259343019618573e-5  …  -1.114952956741447e-7, -3.2087715320026836e-8, -6.298362945829872e-7, -9.907462868556357e-8, -7.040814842480976e-7, -1.9945092441430586e-8, -3.0348804932137297e-7, -3.234793338700408e-7, -1.536976961439118e-7, -1.5459767335048958e-7], [0.0 -0.10834159685648033; 0.0 -0.045887056976095; … ; 0.0 -1.536976961439118e-7; 0.0 -1.5459767335048958e-7], 1)

Néanmoins, l'algorithme converge plus rapidement (en temps) en utilisant le facteur de Cholesky directement plutôt que $M$.

In [85]:
using BenchmarkTools

In [86]:
@benchmark pcg_quadratic(A, b, x0, M, trace = true)

BenchmarkTools.Trial: 41 samples with 1 evaluation per sample.
 Range (min … max):   77.807 ms … 189.105 ms  ┊ GC (min … max): 2.43% … 11.74%
 Time  (median):     124.848 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   122.606 ms ±  25.457 ms  ┊ GC (mean ± σ):  1.36% ±  2.63%

          ▁ █ ▁      ▁     █▁    ▁   ▄      ▁                    
  ▆▁▁▆▁▆▁▁█▆█▁█▁▆▆▆▁▁█▁▁▆▆▁██▆▆▁▁█▁▆▁█▆▁▆▁▁▆█▆▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▆ ▁
  77.8 ms          Histogram: frequency by time          189 ms <

 Memory estimate: 15.40 MiB, allocs estimate: 58.

In [87]:
@benchmark pcg_quadratic(A, b, x0, C.L, trace = true)

BenchmarkTools.Trial: 431 samples with 1 evaluation per sample.
 Range (min … max):   4.776 ms … 49.135 ms  ┊ GC (min … max):  0.00% … 74.64%
 Time  (median):     10.570 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   11.569 ms ±  4.991 ms  ┊ GC (mean ± σ):  14.67% ± 18.04%

     ▁ ▂ ▆▆█▂▆▄▂▄▃▁▄▂ ▂ ▂                                      
  ▄▄▆█▆████████████████▆█▇▄▄▆▄▆▄▄▃▄▄▄▃▃▁▂▃▃▁▁▃▁▁▁▂▂▁▁▁▁▁▂▂▂▁▃ ▄
  4.78 ms         Histogram: frequency by time        28.9 ms <

 Memory estimate: 7.77 MiB, allocs estimate: 56.

Travailler avec le facteur de Cholesky accélère grandement la résolution du système!

Travaillons à présent avec la factorisation incomplète.

In [88]:
C = ichol(A)

1000×1000 LowerTriangular{Float64, Matrix{Float64}}:
 1.73205   ⋅         ⋅         ⋅       …     ⋅           ⋅            ⋅ 
 0.57735  2.38048    ⋅         ⋅             ⋅           ⋅            ⋅ 
 0.0      0.420084  3.28991    ⋅             ⋅           ⋅            ⋅ 
 0.0      0.0       0.303959  4.23174        ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.23631        ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0      …     ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0      …     ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅

La factorisation reste incomplète, comme illustré ci-dessous.

In [89]:
M=C*C'
norm(M-A)

0.47140452079103184

Nous pouvons néanmoins utiliser ce facteur pour résoudre le système.

In [90]:
x, iter, k = pcg_quadratic(A, b, x0, M, trace = true)

([-0.1083415973576645, -0.04588705614477204, -0.005969577399261489, -0.03808376595407495, -0.00215386504210714, -0.007235570892163519, -0.007876434057330696, -0.012674365065675982, -0.0018485519458593808, -5.2593430291504795e-5  …  -1.1149529587621608e-7, -3.208771537818182e-8, -6.29836295724487e-7, -9.907462886512403e-8, -7.040814855241578e-7, -1.9945092477578587e-8, -3.0348804987140737e-7, -3.2347933445630687e-7, -1.5369769642246786e-7, -1.545976755970789e-7], [0.0 -0.10834160861259165 -0.1083415973576645; 0.0 -0.04588704924789038 -0.04588705614477204; … ; 0.0 -1.5369771654401708e-7 -1.5369769642246786e-7; 0.0 -1.3930201778315342e-7 -1.545976755970789e-7], 2)

In [91]:
k

2

Nous prenons à présent 2 itérations, comme la factorisation n'a pas été conduite jusqu'au bout.

In [92]:
@benchmark pcg_quadratic(A, b, x0, M, trace = true)

BenchmarkTools.Trial: 36 samples with 1 evaluation per sample.
 Range (min … max):  100.319 ms … 236.641 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     137.059 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   139.576 ms ±  22.532 ms  ┊ GC (mean ± σ):  1.80% ± 3.27%

        ▁ ▁ █ ▄  ▄█▁▁ ▁ ▁▁                                       
  ▆▁▁▁▁▁█▁█▁█▁█▁▆████▆█▆██▁▁▁▆▆▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  100 ms           Histogram: frequency by time          237 ms <

 Memory estimate: 23.13 MiB, allocs estimate: 94.

In [93]:
@benchmark pcg_quadratic(A, b, x0, C, trace = true)

BenchmarkTools.Trial: 843 samples with 1 evaluation per sample.
 Range (min … max):  2.742 ms … 34.205 ms  ┊ GC (min … max): 0.00% … 87.38%
 Time  (median):     5.637 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.878 ms ±  1.860 ms  ┊ GC (mean ± σ):  0.60% ±  3.01%

           ▁▁▂▁▂▄▄█▂▇▁▅▄▂▅▃▁▃▇ ▃                              
  ▁▂▃▂▄▇▇▇▇███████████████████▆█▇█▇▄▅▆▅▄▄▆▄▃▃▄▂▃▂▂▃▂▂▁▁▂▁▂▂▃ ▄
  2.74 ms        Histogram: frequency by time        10.6 ms <

 Memory estimate: 244.75 KiB, allocs estimate: 85.

Travailler avec la matrice sous forme plein ralentit le code en raison du nombre plus grand d'itérations, mais la version triangulaire est encore plus rapide.

Mais dans tous les cas, nous sommes bien plus efficace que la version non préconditionnée.

In [94]:
M = zeros(n,n)+I
@benchmark pcg_quadratic(A, b, x0, M, trace = true)

BenchmarkTools.Trial: 1 sample with 1 evaluation per sample.
 Single result which took 5.500 s (16.06% GC) to evaluate,
 with a memory estimate of 3.82 GiB, over 33052 allocations.

Notons aussi que la version proposée du gradient conjuguée, avec le préconditioneur triangulaire, est plus rapide que la résolution directe.

In [95]:
@benchmark A\(-b)

BenchmarkTools.Trial: 86 samples with 1 evaluation per sample.
 Range (min … max):  31.817 ms … 97.619 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     55.916 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   58.583 ms ± 15.807 ms  ┊ GC (mean ± σ):  1.18% ± 3.62%

       ▃        ▃      █ ▁      ▁                              
  ▆▄▁▄▁█▆▆▁▄▄▆▁▇█▆▁▁▄▇▆█▆█▆▄▆▁▁▆█▇▄▄▄▄▄▆▆▁▆▆▆▁▁▁▄▁▄▁▄▁▆▄▁▄▄▁▄ ▁
  31.8 ms         Histogram: frequency by time        93.7 ms <

 Memory estimate: 7.65 MiB, allocs estimate: 12.

Évidemment, ceci néglige le temps utlisé pour calculer le facteur. Malheureusement, le code implémenté n'est pas encore très efficace...

In [96]:
@benchmark ichol(A)

BenchmarkTools.Trial: 18 samples with 1 evaluation per sample.
 Range (min … max):  244.247 ms … 415.278 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     265.010 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   283.950 ms ±  48.239 ms  ┊ GC (mean ± σ):  0.11% ± 0.51%

  ▃▃     █                                                       
  ██▁▇▇▇▇█▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  244 ms           Histogram: frequency by time          415 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 4.

Une implémentation efficace fait usage des matrices creuses et de fonctions spécifiques pour caluler $v$.

Commençons avec une version simple.

In [97]:
using SparseArrays

"""
    ichol_sparse(A::SparseMatrixCSC; drop_tol=0.0)

Factorisation incomplète de Cholesky optimisée pour matrices creuses.

# Arguments
- `A::SparseMatrixCSC`: Matrice creuse symétrique définie positive
- `drop_tol::Float64`: Seuil pour éliminer les petits éléments (0.0 = pas de drop)

# Returns
- Facteur de Cholesky incomplet L tel que A ≈ L*L'
"""
function ichol_sparse(A::SparseMatrixCSC{Float64, Int}; drop_tol=0.0)
    n = size(A, 1)
    
    # Copier la partie triangulaire inférieure de A
    L = tril(A)
    
    # Récupérer les structures de données CSC pour un accès rapide
    rows = rowvals(L)
    vals = nonzeros(L)
    
    for k = 1:n
        # Trouver la position de L[k,k] dans la structure CSC
        col_range = nzrange(L, k)
        
        # Calculer L[k,k]
        diag_idx = findfirst(i -> rows[col_range[i]] == k, 1:length(col_range))
        if diag_idx === nothing
            error("Élément diagonal manquant en position ($k,$k)")
        end
        diag_pos = col_range[diag_idx]
        
        if vals[diag_pos] <= 0
            error("Matrice non définie positive à la position ($k,$k)")
        end
        
        vals[diag_pos] = sqrt(vals[diag_pos])
        L_kk = vals[diag_pos]
        
        # Diviser la colonne k par L[k,k]
        for idx in col_range
            if rows[idx] > k
                vals[idx] /= L_kk
            end
        end
        
        # Mise à jour des colonnes suivantes
        for idx_k in col_range
            i = rows[idx_k]
            if i > k
                L_ik = vals[idx_k]
                
                # Chercher la colonne i et mettre à jour
                col_i_range = nzrange(L, i)
                
                for idx_i in col_i_range
                    j = rows[idx_i]
                    if j >= i
                        # Trouver L[j,k] si il existe
                        L_jk = 0.0
                        for idx_search in col_range
                            if rows[idx_search] == j
                                L_jk = vals[idx_search]
                                break
                            end
                        end
                        
                        # Mise à jour: L[j,i] -= L[j,k] * L[i,k]
                        vals[idx_i] -= L_jk * L_ik
                        
                        # Drop des petits éléments
                        if abs(vals[idx_i]) < drop_tol
                            vals[idx_i] = 0.0
                        end
                    end
                end
            end
        end
    end
    
    # Nettoyer les zéros si drop_tol > 0
    if drop_tol > 0
        dropzeros!(L)
    end
    
    return LowerTriangular(L)
end

ichol_sparse

Essayons avec notre matrice $A$, que nous modifions initialement pour la mettre sous forme creuse.

In [98]:
S = sparse(A)
C = ichol_sparse(S)

1000×1000 LowerTriangular{Float64, SparseMatrixCSC{Float64, Int64}}:
 1.73205   ⋅         ⋅         ⋅       …     ⋅           ⋅            ⋅ 
 0.57735  2.38048    ⋅         ⋅             ⋅           ⋅            ⋅ 
 0.0      0.420084  3.28991    ⋅             ⋅           ⋅            ⋅ 
 0.0      0.0       0.303959  4.23174        ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.23631        ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0      …     ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0      …     ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅         

Avons-nous gagné en performance?

In [99]:
@benchmark ichol_sparse(S)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  34.700 μs …  28.046 ms  ┊ GC (min … max): 0.00% … 99.55%
 Time  (median):     64.300 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   83.825 μs ± 387.332 μs  ┊ GC (mean ± σ):  6.48% ±  1.41%

     ▇█                                                         
  █▅▄██▇▅▄▅▄▃▃▄▅▅▄▄▃▄▅▅▅▄▃▃▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  34.7 μs         Histogram: frequency by time          230 μs <

 Memory estimate: 39.33 KiB, allocs estimate: 9.

La méthode est cette-fois nettement plus rapide!

Mais nous pouvons produire une implémentation plus robuste.

In [100]:
"""
    ichol_ikj(A::SparseMatrixCSC; drop_tol=0.0)

Factorisation incomplète de Cholesky avec ordre des boucles IKJ
(plus cache-friendly et permet vectorisation).
"""
function ichol_ikj(A::SparseMatrixCSC{Float64, Int}; drop_tol=0.0)
    n = size(A, 1)
    L = tril(A)
    
    rows = rowvals(L)
    vals = nonzeros(L)
    
    # Vecteur de travail pour stocker la colonne courante
    work = zeros(n)
    
    for k = 1:n
        # Copier la colonne k dans work
        fill!(work, 0.0)
        for idx in nzrange(L, k)
            i = rows[idx]
            work[i] = vals[idx]
        end
        
        # Calculer L[k,k]
        if work[k] <= 0
            error("Matrice non définie positive à ($k,$k)")
        end
        work[k] = sqrt(work[k])
        
        # Normaliser la colonne
        for i = (k+1):n
            if abs(work[i]) > drop_tol
                work[i] /= work[k]
            else
                work[i] = 0.0
            end
        end
        
        # Réécrire la colonne k
        col_k_range = nzrange(L, k)
        idx = 1
        for pos in col_k_range
            i = rows[pos]
            vals[pos] = work[i]
            idx += 1
        end
        
        # Mettre à jour les colonnes suivantes
        for i = (k+1):n
            if abs(work[i]) > drop_tol
                L_ik = work[i]
                
                # Mise à jour de la colonne i
                for idx in nzrange(L, i)
                    j = rows[idx]
                    if j >= i
                        vals[idx] -= L_ik * work[j]
                    end
                end
            end
        end
    end
    
    if drop_tol > 0
        dropzeros!(L)
    end
    
    return LowerTriangular(L)
end

ichol_ikj

@benchmark ichol_sparse(S)

Une variante consiste à permettre de définir un pattern de remplissage.

In [101]:
"""
    ichol_ic0(A::Matrix)

Factorisation incomplète de Cholesky IC(0).
Le pattern de L est identique à celui de tril(A).
"""
function ichol_ic0(A::Matrix)
    n = size(A, 1)
    L = zeros(n, n)
    
    # Copier la partie triangulaire inférieure de A
    for i = 1:n
        for j = 1:i
            L[i, j] = A[i, j]
        end
    end
    
    for k = 1:n
        # L[k,k] = sqrt(L[k,k])
        if L[k, k] <= 0
            error("Matrice non définie positive à la ligne $k")
        end
        L[k, k] = sqrt(L[k, k])
        
        # Normaliser la colonne k
        for i = (k+1):n
            if L[i, k] != 0  # Respecter le pattern
                L[i, k] = L[i, k] / L[k, k]
            end
        end
        
        # Mettre à jour les colonnes j > k
        for j = (k+1):n
            if L[j, k] != 0  # Si L[j,k] existe
                for i = j:n
                    if L[i, j] != 0  # Si L[i,j] existe dans le pattern original
                        L[i, j] = L[i, j] - L[i, k] * L[j, k]
                    end
                end
            end
        end
    end
    
    return LowerTriangular(L)
end

ichol_ic0

In [102]:
C = ichol_ic0(A)

1000×1000 LowerTriangular{Float64, Matrix{Float64}}:
 1.73205   ⋅         ⋅         ⋅       …     ⋅           ⋅            ⋅ 
 0.57735  2.38048    ⋅         ⋅             ⋅           ⋅            ⋅ 
 0.0      0.420084  3.28991    ⋅             ⋅           ⋅            ⋅ 
 0.0      0.0       0.303959  4.23174        ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.23631        ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0      …     ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0      …     ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅ 
 0.0      0.0       0.0       0.0            ⋅           ⋅            ⋅

In [103]:
@benchmark ichol_ic0(A)

BenchmarkTools.Trial: 321 samples with 1 evaluation per sample.
 Range (min … max):   6.361 ms … 48.930 ms  ┊ GC (min … max): 0.00% … 57.44%
 Time  (median):     14.616 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.531 ms ±  5.927 ms  ┊ GC (mean ± σ):  7.51% ± 11.07%

     █  ▁   ▃▃▄▂▄▇▃▂ ▄ ▄▁▄ ▁                                   
  ▆▅▄█▅██▇████████████▇███▆█▆█▆▃█▃▆▆▅▄▃▃▃▃▁▃▃▃▃▁▃▁▁▁▁▁▁▁▁▁▃▁▃ ▄
  6.36 ms         Histogram: frequency by time        35.9 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 4.

On va ajuster notre méthode de gradient conjugué préconditionné pour accepter les matrices creuses.

In [104]:
"""
    pcg_quadratic(A, b, x0, L; trace=false, tol=1e-6, max_iter=nothing)

Gradient conjugué préconditionné pour minimiser f(x) = ½x'Ax - b'x.

# Arguments
- `A`: Matrice (dense ou creuse) symétrique définie positive
- `b::Vector`: Vecteur du second membre
- `x0::Vector`: Solution initiale
- `L`: Facteur de Cholesky du préconditionneur (LowerTriangular, dense ou creux)
- `trace::Bool`: Si true, retourne l'historique des itérations
- `tol::Float64`: Tolérance sur le résidu
- `max_iter::Int`: Nombre maximum d'itérations (défaut: length(x0))

# Returns
- `x::Vector`: Solution approchée
- `k::Int`: Nombre d'itérations
- `iter::Matrix`: Historique (si trace=true)
"""
function pcg_quadratic(A::Union{Matrix{T}, SparseMatrixCSC{T}}, 
                       b::Vector{T}, 
                       x0::Vector{T}, 
                       L::Union{LowerTriangular{T, Matrix{T}}, 
                               LowerTriangular{T, SparseMatrixCSC{T, Int}}};
                       trace::Bool = false, 
                       tol::T = T(1e-6),
                       max_iter::Union{Int, Nothing} = nothing) where T <: AbstractFloat
    
    n = length(x0)
    max_iter = max_iter === nothing ? n : max_iter
    
    # Initialisation
    x = copy(x0)  # Copie pour ne pas modifier x0
    
    if trace
        iter = zeros(T, n, max_iter + 1)
        iter[:, 1] = x
    end
    
    # Gradient initial: ∇f(x) = Ax - b, donc g = b + Ax pour notre formulation
    g = b + A * x
    
    # Préconditionneur: M = L*L', donc M\g = L'\(L\g)
    U = L'
    v = U \ (L \ g)
    d = -v
    
    k = 0
    tol2 = tol * tol
    gv = dot(g, v)
    
    # Vecteur temporaire pour éviter allocations
    Ad = similar(x)
    g_temp = similar(x)
    v_temp = similar(x)
    
    while (gv > tol2) && (k < max_iter)
        # Ad = A * d (réutilise le vecteur existant)
        mul!(Ad, A, d)
        
        normd = dot(d, Ad)
        α = gv / normd
        
        # x += α * d
        axpy!(α, d, x)  # x = x + α*d (BLAS optimisé)
        
        if trace && k + 1 < size(iter, 2)
            iter[:, k + 2] = x
        end
        
        # g += α * Ad
        axpy!(α, Ad, g)
        
        # v = U \ (L \ g)
        ldiv!(g_temp, L, g)
        ldiv!(v, U, g_temp)
        
        gvold = gv
        gv = dot(g, v)
        β = gv / gvold
        
        # d = -v + β * d
        @. d = -v + β * d  # Vectorisé, évite allocation
        
        k += 1
    end
    
    if trace
        return x, iter[:, 1:(k+1)], k
    end
    return x, k
end

# Surcharge pour accepter juste une matrice (sans préconditionneur)
"""
    pcg_quadratic(A, b, x0; kwargs...)

Version sans préconditionneur (utilise M = I).
"""
function pcg_quadratic(A::Union{Matrix{T}, SparseMatrixCSC{T}}, 
                       b::Vector{T}, 
                       x0::Vector{T};
                       trace::Bool = false, 
                       tol::T = T(1e-6),
                       max_iter::Union{Int, Nothing} = nothing) where T <: AbstractFloat
    
    n = length(x0)
    max_iter = max_iter === nothing ? n : max_iter
    
    x = copy(x0)
    
    if trace
        iter = zeros(T, n, max_iter + 1)
        iter[:, 1] = x
    end
    
    # Sans préconditionneur: v = g
    g = b + A * x
    v = copy(g)
    d = -v
    
    k = 0
    tol2 = tol * tol
    gv = dot(g, v)
    
    Ad = similar(x)
    
    while (gv > tol2) && (k < max_iter)
        mul!(Ad, A, d)
        
        normd = dot(d, Ad)
        α = gv / normd
        
        axpy!(α, d, x)
        
        if trace && k + 1 < size(iter, 2)
            iter[:, k + 2] = x
        end
        
        axpy!(α, Ad, g)
        
        v = copy(g)
        
        gvold = gv
        gv = dot(g, v)
        β = gv / gvold
        
        @. d = -v + β * d
        
        k += 1
    end
    
    if trace
        return x, iter[:, 1:(k+1)], k
    end
    return x, k
end

pcg_quadratic

In [114]:
x, iter, k = pcg_quadratic(S, b, x0, ichol_sparse(S), trace = true)

([-0.10834159735766451, -0.04588705614477203, -0.005969577399261488, -0.038083765954074957, -0.00215386504210714, -0.007235570892163519, -0.007876434057330694, -0.012674365065675982, -0.0018485519458593805, -5.2593430291504775e-5  …  -1.1149529587621607e-7, -3.208771537818182e-8, -6.29836295724487e-7, -9.907462886512404e-8, -7.040814855241576e-7, -1.9945092477578587e-8, -3.0348804987140737e-7, -3.234793344563069e-7, -1.5369769642246786e-7, -1.5459767559707893e-7], [0.0 -0.10834160861259166 -0.10834159735766451; 0.0 -0.04588704924789038 -0.04588705614477203; … ; 0.0 -1.5369771654401708e-7 -1.5369769642246786e-7; 0.0 -1.393020177831534e-7 -1.5459767559707893e-7], 2)

Nous avons à présent une méthode beaucoup plus efficace, tel qu'illustré par le benchmark ci-après.

In [113]:
@benchmark pcg_quadratic(S, b, x0, ichol_sparse(S), trace = true)

BenchmarkTools.Trial: 1025 samples with 1 evaluation per sample.
 Range (min … max):  1.772 ms … 66.428 ms  ┊ GC (min … max):  0.00% … 90.18%
 Time  (median):     3.488 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.832 ms ±  4.018 ms  ┊ GC (mean ± σ):  27.95% ± 25.91%

  ▁▃█▇▄▄▃                                                     
  ███████▇▅▅▅▃▆▄▃▄▅▄▄▄▄▅▄▄▃▃▃▃▃▃▃▂▂▂▂▁▂▁▂▂▁▂▂▂▁▁▁▁▂▂▂▁▂▂▁▁▁▂ ▃
  1.77 ms        Histogram: frequency by time        18.7 ms <

 Memory estimate: 7.76 MiB, allocs estimate: 41.

La précision demeure par ailleurs excellente.

In [116]:
norm(x+A\b)

9.780124711879884e-10

Une version plus robuste de la méthode est donnée ci-dessous.

In [111]:
"""
    pcg_quadratic_robust(A, b, x0, L; trace=false, tol=1e-6, max_iter=nothing)

Version plus robuste qui calcule le vrai résidu périodiquement.
"""
function pcg_quadratic_robust(A::Union{Matrix{T}, SparseMatrixCSC{T}}, 
                              b::Vector{T}, 
                              x0::Vector{T}, 
                              L::Union{LowerTriangular{T, Matrix{T}}, 
                                      LowerTriangular{T, SparseMatrixCSC{T, Int}}};
                              trace::Bool = false, 
                              tol::T = T(1e-6),
                              max_iter::Union{Int, Nothing} = nothing,
                              restart_freq::Int = 50) where T <: AbstractFloat
    
    n = length(x0)
    max_iter = max_iter === nothing ? n : max_iter
    
    x = copy(x0)
    
    if trace
        iter = zeros(T, n, max_iter + 1)
        residuals = zeros(T, max_iter + 1)
        iter[:, 1] = x
    end
    
    # Résidu: r = b - Ax pour le système Ax = b
    # Mais ici on minimise ½x'Ax - b'x, donc ∇f = Ax - b
    # Le résidu du système associé est r = b - Ax
    r = b - A * x
    
    if trace
        residuals[1] = norm(r)
    end
    
    U = L'
    z = U \ (L \ r)  # z = M^{-1} r
    p = copy(z)
    
    k = 0
    tol2 = tol * tol
    rz = dot(r, z)
    
    Ap = similar(x)
    r_temp = similar(x)
    z_temp = similar(x)
    
    while (rz > tol2) && (k < max_iter)
        mul!(Ap, A, p)
        
        pAp = dot(p, Ap)
        α = rz / pAp
        
        # x = x + α * p
        axpy!(α, p, x)
        
        # r = r - α * Ap
        axpy!(-α, Ap, r)
        
        # Recalcul périodique du vrai résidu (évite accumulation d'erreurs)
        if k % restart_freq == 0
            r = b - A * x
        end
        
        if trace && k + 1 <= max_iter
            iter[:, k + 2] = x
            residuals[k + 2] = norm(r)
        end
        
        # z = M^{-1} r
        ldiv!(r_temp, L, r)
        ldiv!(z, U, r_temp)
        
        rz_old = rz
        rz = dot(r, z)
        β = rz / rz_old
        
        # p = z + β * p
        @. p = z + β * p
        
        k += 1
    end
    
    if trace
        return x, iter[:, 1:(k+1)], residuals[1:(k+1)], k
    end
    return x, k
end

pcg_quadratic_robust

In [115]:
@benchmark pcg_quadratic_robust(S, b, x0, ichol_sparse(S), trace = true)

BenchmarkTools.Trial: 933 samples with 1 evaluation per sample.
 Range (min … max):  1.844 ms … 36.965 ms  ┊ GC (min … max):  0.00% … 90.66%
 Time  (median):     4.141 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.307 ms ±  3.768 ms  ┊ GC (mean ± σ):  27.99% ± 26.31%

  ▁▅▇▇█▅▅▄▁▅▁▂▃                                               
  █████████████▆▅▆▄▄▄▅▄▃▄▄▃▃▃▃▃▃▃▃▄▄▂▃▃▂▃▃▃▃▂▃▂▃▂▃▁▂▂▃▁▁▂▂▁▂ ▄
  1.84 ms        Histogram: frequency by time        17.9 ms <

 Memory estimate: 7.78 MiB, allocs estimate: 52.

In [118]:
x, iter, k = pcg_quadratic(S, b, x0, ichol_sparse(S), trace = true)
norm(x+A\b)

9.780124711879884e-10